# 1. Kaggle Datasets

## 1.1. What is Kaggle?

## 1.2. Getting Started

### Sign up


### Create an API Token

![User settings menu](figures/kaggle_my_account.png)

Scroll down a bit and you will find an API section. Click '`Create New API Token`' button.

![Create New Kaggle API Token](figures/create_new_kaggle_api_token.png)


### Copy JSON file

In [1]:
# !pip uninstall kaggle
!pip install kaggle

In [7]:
import os
from pathlib import Path
home = str(Path.home())
kaggle_root = os.path.join(*[home, '.kaggle'])
os.mkdir(kaggle_root)

In [8]:
import json
token = {"username":"bigdata","key":"abcd1234"}
with open(os.path.join(*[kaggle_root, 'kaggle.json']), 'w') as file:
    json.dump(token, file)

In [26]:
!kaggle datasets download -d neuromusic/avocado-prices -p data


  0%|          | 0.00/629k [00:00<?, ?B/s]
100%|##########| 629k/629k [00:00<00:00, 17.2MB/s]


In [27]:
import platform
if platform.system() == 'Windows':
    this_file_path = !echo %cd%
    data_path = os.path.join(*[this_file_path[0], 'data'])
    print('[IMPORTANT] No automatic unzipping supported on Windows.')
    print('You have to open `File Explorer` and manually unzip `' + data_path + '\\avocado-prices.zip`')
    print('Make sure `avocado.csv` file in the zip file is placed directly under `data` folder:')
    print('|- ica02')
    print('    |- data')
    print('        |- avocado.csv')
    print('    |- How_to_Read_and_Represent_Data.ipynb')
else:
    !unzip ./data/new-york-city-current-job-postings.zip ./data

[IMPORTANT] No automatic unzipping supported on Windows.
You have to open `File Explorer` and manually unzip `E:\dev\bigdata\in-class-assignments\ica02\data\avocado-prices.zip`
Make sure `avocado.csv` file in the zip file is placed directly under `data` folder:
|- ica02
    |- data
        |- avocado.csv
    |- How_to_Read_and_Represent_Data.ipynb


'Expand-Archive' is not recognized as an internal or external command,
operable program or batch file.
